In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src')) # or the path to your source code
sys.path.insert(0, module_path)
import pandas
from query import query_cdp_data as query
from enrich import enrich_cdp_data as enrich
from ingest_bulk import ingest_cdp_data_bulk as ingest_bulk

In [ ]:
event = {}
event['login_url'] = 'https://login.salesforce.com' 
event['user_name'] = '' # CDP org username
event['password'] = '' # CDP org password
event['client_id'] = '' # CDP connected app client id
event['client_secret'] = '' # CDP connected app client secret
event['dlo_name'] = 'athlete_profiles' #CDP org's DLO name
event['dlo_source_name'] = 'athlete_api' #CDP org's Ingestion API source name 
event['dlo_object'] = 'athlete_api_athlete_profiles_E6B17A31__dll' #DLO object name to be queried
event['dlo_filter'] = 'limit 100' #Where clause for the query
event['bulk_operation_type'] = 'upsert' #Where clause for the query

In [ ]:
# Query DLO
df = query(event)
print(len(df))
df.head()

In [ ]:
# Enrich DLO
df = enrich(df)

In [ ]:
# Bulk update Enriched DLO
ingest_bulk(event,df)